[View in Colaboratory](https://colab.research.google.com/github/rajagopalmotivate/DeepLearnIITMA1A-Visualize/blob/master/Visualize_Filter.ipynb)

In [1]:
!wget "https://github.com/rajagopalmotivate/DeepLearnIITMA1A-Visualize/raw/master/cifar_1B_softmax_model9021s.h5"


Redirecting output to ‘wget-log’.


In [2]:
from keras import backend as K


Using TensorFlow backend.


In [0]:
import tensorflow as tf
from keras import Input
from keras import layers
from keras import datasets
import numpy as np
from keras import utils
from keras.models import  Model
from keras.utils import plot_model
import keras
import os
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.models import model_from_json

from sklearn.metrics import classification_report
from decimal import Decimal
import  matplotlib.pyplot as plt
from scipy.misc import toimage
from keras.preprocessing.image import ImageDataGenerator
import copy
import numpy as np

import math as maths
import matplotlib.pyplot as plt
from keras.models import load_model


In [5]:
myloadedmodeloriginal  = load_model('cifar_1B_softmax_model9021s.h5')


ValueError: ignored

In [0]:
#Find insights about a network such as how deep is the network, memorizationcapacity, generalizationcapacity, noofCNNs, kernalsize, amountofnonlinearity
def extractNetworkInsights(anetwork, verbose=0):
    #print('Analzing the Network insights')
    layercount = 0
    layercountCNN = 0
    for alayer in anetwork.layers:
      layercount = layercount + 1
      aconfig = alayer.get_config()
      if (verbose==0):
        print(str(aconfig) )
      if (verbose==1):  
        if 'CNN' in str(aconfig.get("name")):
          layercountCNN = layercountCNN + 1 
          print( str(layercount) + '   ' + str(aconfig.get("name") +  '   Filters=' + str(aconfig.get("filters") )))
        elif ('Pool' in str(aconfig.get("name"))):
          print( str(layercount) + '   ' + str(aconfig.get("name") +  '   PoolSize=' + str(aconfig.get("pool_size") )))
        elif 'dropout' in aconfig.get("name"):
          print( str(layercount) + '   ' + str(aconfig.get("name") +  '   Rate=' + str(aconfig.get("rate") )))
        elif 'Dense' in aconfig.get("name"):
          print( str(layercount) + '   ' + str(aconfig.get("name") +  '   Units=' + str(aconfig.get("units") )) )         
        else:
          print( str(layercount) + '   ' + str(aconfig.get("name")))
    return str(layercountCNN) + ' CNN layers'

In [0]:
extractNetworkInsights(myloadedmodeloriginal, verbose=1)

In [0]:
import pandas
from pandas import DataFrame
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
import matplotlib.cm as mpcm
from keras import backend as K
from keras import models

In [0]:
(xtrain1, ytrain1), (xtest, ytest) = datasets.cifar10.load_data()
labelsnames =  ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']


In [0]:
def plotAllFeatureResponsesForaImageByaLayer(myloadedmodel, myimage , layerno):
    imgtensor = myimage.copy()
    imgtensor = np.expand_dims(imgtensor, axis=0)
    imgtensor = imgtensor * 1.0/255.0
  #  plt.imshow(imgtensor[0])
    nooflayersinthemodel = len(myloadedmodel.layers)
    mylayeroutputs = [layer.output for layer in myloadedmodel.layers[:nooflayersinthemodel]]
    myactivationmodel = models.Model(inputs=myloadedmodel.input, outputs = mylayeroutputs)
    myactivations = myactivationmodel.predict(imgtensor)
    for layeriterate in range(layerno-4 , layerno + 1):
        if(((layerno + 1 -layeriterate) > 0) & (layeriterate>=0)):
            print( str(layeriterate) + '  :' + str(myloadedmodel.layers[layeriterate].name))
    firstlayeractivation =myactivations[layerno]
    print('Layer no '+ str(layerno) + ' of ' + str(nooflayersinthemodel) + ' layer model has ' + str(firstlayeractivation.shape[3]) + ' filters. Here is the feature response for these filters for this input image')
    nooffilters = firstlayeractivation.shape[3]
    plotrows = int( maths.sqrt(nooffilters) )
    plotrows = plotrows + 1
    
    fig=plt.figure(figsize=(16, 16))
    for i in range(firstlayeractivation.shape[3]):
        fig.add_subplot(plotrows, plotrows, i+1)
        plt.imshow(firstlayeractivation[0,:,:,i])
        plt.title(str(i), color='C5')
        plt.axis('off')
    plt.show()


    #plt.matshow(firstlayeractivation[0,:,:,1])

In [0]:
def islabelmatchDirectlybyLabel(thisloadedmodel, labelno, myunnormalizedimage):
    ismatched = False
    myimageJ2 = myunnormalizedimage.copy()
    myimageJnormalized2 = myimageJ2.astype('float16') * 1/255
    imageforprediction2 = np.expand_dims(myimageJnormalized2, axis=0)
    result2 = thisloadedmodel.predict(imageforprediction2, batch_size=1)
    y_classes2 = result2.argmax(axis=-1)  
    predictedlabel2 = y_classes2[0]
    #print(predictedlabel2)
    if(predictedlabel2 == labelno):
        ismatched = True
    return ismatched 

In [0]:
def SearchByLabelFeatureResponsesForaImageSetForaFilterInaLayer(myloadedmodel, labelno, imagedataset, imagestartindex , batchsize, layerno, filterno):

    nooflayersinthemodel = len(myloadedmodel.layers)
    mylayeroutputs = [layer.output for layer in myloadedmodel.layers[:nooflayersinthemodel]]
    myactivationmodel = models.Model(inputs=myloadedmodel.input, outputs = mylayeroutputs)
    plotimagecount = 1
    
    
    fig=plt.figure(figsize=(16, 16))
    for i in range(batchsize-4):
        imgtensor = imagedataset[imagestartindex+i].copy()
        ismatching = islabelmatchDirectlybyLabel(myloadedmodel, labelno, imgtensor)
        if(ismatching==True):
            imgtensor = np.expand_dims(imgtensor, axis=0)
            imgtensor = imgtensor * 1.0/255.0
            fig.add_subplot(16, 16, plotimagecount)
            plt.imshow(imgtensor[0])
            plt.axis('off')
            myactivations = myactivationmodel.predict(imgtensor)
            firstlayeractivation =myactivations[layerno]
            fig.add_subplot(16, 16, plotimagecount+1)
            plt.imshow(firstlayeractivation[0,:,:,filterno])
            plt.axis('off')
            plotimagecount = plotimagecount +2 
            if (plotimagecount >= 16*16):
                break
    plt.show()


In [0]:
def FindimagesthatresponsmosttothisFilter(myloadedmodel, labelno, imagedataset, imagestartindex , batchsize, layerno, filterno):

    nooflayersinthemodel = len(myloadedmodel.layers)
    mylayeroutputs = [layer.output for layer in myloadedmodel.layers[:nooflayersinthemodel]]
    myactivationmodel = models.Model(inputs=myloadedmodel.input, outputs = mylayeroutputs)
    plotimagecount = 1
    
    mylist = list()
    
    fig=plt.figure(figsize=(16, 16))
    for i in range(batchsize-4):
        imgtensor = imagedataset[imagestartindex+i].copy()
        ismatching = islabelmatchDirectlybyLabel(myloadedmodel, labelno, imgtensor)
        ismatching = True
        if(ismatching==True):
            imgtensor = np.expand_dims(imgtensor, axis=0)
            imgtensor = imgtensor * 1.0/255.0
            fig.add_subplot(16, 16, plotimagecount)
            plt.imshow(imgtensor[0])
            plt.axis('off')
            myactivations = myactivationmodel.predict(imgtensor)
            firstlayeractivation =myactivations[layerno]
            fig.add_subplot(16, 16, plotimagecount+1)
            plt.imshow(firstlayeractivation[0,:,:,filterno])
            myactivationfeaturemap = firstlayeractivation[0,:,:,filterno]
            myactivationlevel = myactivationfeaturemap.mean()
            myactivationlevel = (myactivationlevel * 100).round(2)
            
            elem = (imagestartindex+i, myactivationlevel)
            mylist.append(elem)
            #print(myactivationlevel)
            plt.axis('off')
            plotimagecount = plotimagecount +2 
            if (plotimagecount >= 16*16-1):
                    plotimagecount = 1

    plt.show()

    return mylist

In [0]:
def Showimages(myloadedmodel, imagedataset, mylistofimages , layerno, filterno):

    nooflayersinthemodel = len(myloadedmodel.layers)
    mylayeroutputs = [layer.output for layer in myloadedmodel.layers[:nooflayersinthemodel]]
    myactivationmodel = models.Model(inputs=myloadedmodel.input, outputs = mylayeroutputs)
    plotimagecount = 1
    
    
    fig=plt.figure(figsize=(16, 16))
    for i in range(len(mylistofimages)):
        imageindex = mylistofimages[i][0]
        imgtensor = imagedataset[imageindex].copy()
        imgtensor = np.expand_dims(imgtensor, axis=0)
        imgtensor = imgtensor * 1.0/255.0
        fig.add_subplot(10, 10, plotimagecount)
        plt.imshow(imgtensor[0])
        plt.axis('off')
        myactivations = myactivationmodel.predict(imgtensor)
        firstlayeractivation =myactivations[layerno]
        fig.add_subplot(10, 10, plotimagecount+1)
        plt.imshow(firstlayeractivation[0,:,:,filterno])
        plt.axis('off')
        plotimagecount = plotimagecount +2 
        if (plotimagecount >= 10*10):
            break
    plt.show()

In [0]:
from operator import itemgetter, attrgetter

mylist = FindimagesthatresponsmosttothisFilter(myloadedmodeloriginal, 1, xtest, 10, 2000, 15, 43)
sortedlist = sorted(mylist, key=itemgetter(1), reverse=True)


In [0]:
Showimages(myloadedmodeloriginal, xtest, sortedlist , 15, 43)
